### Create key-pair for secure connection

In [1]:
import boto3

In [ ]:
ec2 = boto3.client('ec2', region_name = "us-east-2")

ec2.describe_instances()

response = ec2.create_key_pair(KeyName = "")
response

In [ ]:
# Save pem file
file = open('', "w")
file.write(response["KeyMaterial"])
file.close()

### Create an Amazon EC2 instance

In [ ]:
ec2.describe_instances()

In [ ]:
res = ec2.run_instances(
    ImageId = "",
    MinCount = 1, # number of instances
    MaxCount = 1,
    InstanceType = "t2.micro",
    KeyName = "",
    BlockDeviceMappings = [ # elastic block store
        {
            "DeviceName": "/dev/xvda",
            "Ebs":{
                "DeleteOnTermination": True,
                "VolumeSize": 20

            }
        }
    ],
    TagSpecifications = [
        {
            "ResourceType": "instance",
            "Tags":[
                {
                    "Key": "Name",
                    "Value": "" # name of the instance
                }
            ]
        }
    ]
)

In [ ]:
res

In [ ]:
ec2.describe_instances()

### Create Custome security Group

In [ ]:
res = ec2.describe_security_groups()
res = ec2.create_security_group(
    GroupName = "",
    Description = "Security Group For testing"
)

In [13]:
security_group_id = res["GroupId"]

In [ ]:
security_group_id

In [16]:
## Add inbound rules (ip, port, traffic type)
res = ec2.authorize_security_group_ingress(
    GroupId = security_group_id,
    IpPermissions = [
        {
            "IpProtocol": "tcp",
            "FromPort": 22,
            "ToPort" : 22,
            "IpRanges":[{"CidrIp": "0.0.0.0/0"}]
        }
    ]
)


In [ ]:
res

In [25]:
res = ec2.describe_instances()

instance_id = res["Reservations"][0]["Instances"][0]["InstanceId"]

In [31]:
old_grp_id = res["Reservations"][0]["Instances"][0]["SecurityGroups"][0]["GroupId"]

In [ ]:
## Add custom security group
ec2.modify_instance_attribute(
    InstanceId = instance_id,
    Groups = [security_group_id] # to add multiple just use , and add

)

### Start, Stop and Delete EC2 instance

In [39]:
import time

# check instance status
def wait_for_status(instance_id, target_status):
    while True:
        res = ec2.describe_instances(InstanceIds = [instance_id])

        status = res["Reservations"][0]["Instances"][0]["State"]["Name"]

        if status == target_status:
            print(f"Instance is in {target_status} state")
            break

        time.sleep(10)

In [ ]:
# Start instance
def start_instances(instance_id):
    print("EC2 Instance Start")
    ec2.start_instances(InstanceIds = [instance_id])

    wait_for_status(instance_id, 'running')

In [41]:
start_instances(instance_id)

EC2 Instance Start
Instance is in running state


In [ ]:
# Stop instance
def stop_instances(instance_id):
    print("EC2 Instance Stop")
    ec2.stop_instances(InstanceIds = [instance_id])

    wait_for_status(instance_id, 'stopped')

In [43]:
stop_instances(instance_id)

EC2 Instance Start
Instance is in stopped state


In [48]:
# Tarminate instance
def terminate_instances(instance_id):
    print("EC2 Instance Termination")
    ec2.terminate_instances(InstanceIds = [instance_id])

    wait_for_status(instance_id, 'terminated')


In [49]:
terminate_instances(instance_id)

EC2 Instance Termination
Instance is in terminated state
